In [1]:
pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 272 kB 29.1 MB/s 
     |████████████████████████████████| 3.0 MB 54.8 MB/s 


In [2]:
from Bio import SeqIO
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
neg_seq = []
neg_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Human RNA/Positive.txt", "fasta"):
    neg_seq.append(str(seq_record.seq))
    neg_id.append(str(seq_record.id))
pos_seq = []
pos_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Human RNA/Negative.txt", "fasta"):
    pos_seq.append(str(seq_record.seq))
    pos_id.append(str(seq_record.id))
seqs=pos_seq + neg_seq
dfclass = pd.DataFrame({'class' :  np.repeat((1,0), (741, 741))})
y=dfclass['class']
print(seqs)

['...']


In [5]:
Indneg_seq = []
Indneg_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Human RNA/Ind-positive.txt", "fasta"):
    Indneg_seq.append(str(Indseq_record.seq))
    Indneg_id.append(str(Indseq_record.id))
Indpos_seq = []
Indpos_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Human RNA/Ind-negative.txt", "fasta"):
    Indpos_seq.append(str(Indseq_record.seq))
    Indpos_id.append(str(Indseq_record.id))

Indseqs=Indpos_seq + Indneg_seq
Inddfclass = pd.DataFrame({'class' :  np.repeat((1,0), (334, 3340))})
z=Inddfclass['class']
print(Indseqs)

['...']


In [6]:
pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 32.9 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 


In [7]:
import keras_tuner as kt

In [8]:
max_words = 5000
max_len = 350
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(seqs)
sequences = tok.texts_to_sequences(seqs)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [9]:
#Independent Testing
indtok = Tokenizer(num_words=max_words)
indtok.fit_on_texts(Indseqs)
Indsequences = indtok.texts_to_sequences(Indseqs)
Indsequences_matrix = sequence.pad_sequences(Indsequences,maxlen=max_len)
X_train=sequences_matrix
y_train=y
X_test=Indsequences_matrix
y_test=z

In [10]:
import keras_tuner as kt
from tensorflow import keras

In [11]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(hp.Choice('units', [2, 4, 8, 16, 32, 64, 128]),activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',metrics=['accuracy']) 
  return model

In [12]:
tuner1 = kt.RandomSearch(
    build_model,
    overwrite=True,
    objective='val_accuracy',
    max_trials=10,
    directory='./multiclass_classifier/training')

In [13]:
tuner1.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
best_model = tuner1.get_best_models()[0]

Trial 7 Complete [00h 00m 10s]
val_accuracy: 0.997550368309021

Best val_accuracy So Far: 0.9989112615585327
Total elapsed time: 00h 01m 22s


In [14]:
tuner1.results_summary()

Results summary
Results in ./multiclass_classifier/training/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 128
Score: 0.9989112615585327
Trial summary
Hyperparameters:
units: 16
Score: 0.997550368309021
Trial summary
Hyperparameters:
units: 32
Score: 0.997550368309021
Trial summary
Hyperparameters:
units: 8
Score: 0.9970059990882874
Trial summary
Hyperparameters:
units: 64
Score: 0.982580304145813
Trial summary
Hyperparameters:
units: 2
Score: 0.9090909361839294
Trial summary
Hyperparameters:
units: 4
Score: 0.9090909361839294


In [15]:
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (1482, 128)               44928     
                                                                 
 dense_1 (Dense)             (1482, 1)                 129       
                                                                 
Total params: 45,057
Trainable params: 45,057
Non-trainable params: 0
_________________________________________________________________


In [16]:
accr = best_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

115/115 [==============================] - 0s 2ms/step - loss: 0.0501 - accuracy: 0.9989
Test set
  Loss: 0.050
  Accuracy: 0.999


In [17]:
pred=best_model.predict(X_test)
pred1 = np.round_(pred)

from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)

115/115 [==============================] - 0s 1ms/step
MCC:  0.9934698025207963


array([[3336,    4],
       [   0,  334]])

In [18]:
from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)
print(classification_report(y_test, pred1))
tp, fn, fp, tn = confusion_matrix(y_test, pred1).ravel()
print("MCC ---> {0}".format(mc))
print("Confusion Matrix. tn, fp, fn, tp ---> ", tp, fn, fp, tn)
print("Precision --->TP/TP+Fp ", tp/(tp+fp))
print("Recall - SN - Sensitivity --->TP/TP+FN ", tp/(tp+fn))
print("Specificity - SP ---> ", tn/(tn+fp))
print("Balanced Accuracy ---> ", ((tp/(tp+fn))+(tn/(tn+fp)))/2)
print("Jaccard Index --->TP/TP+FN+FP ", tp/(tp+fn+fp))

MCC:  0.9934698025207963
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3340
           1       0.99      1.00      0.99       334

    accuracy                           1.00      3674
   macro avg       0.99      1.00      1.00      3674
weighted avg       1.00      1.00      1.00      3674

MCC ---> 0.9934698025207963
Confusion Matrix. tn, fp, fn, tp --->  3336 4 0 334
Precision --->TP/TP+Fp  1.0
Recall - SN - Sensitivity --->TP/TP+FN  0.9988023952095808
Specificity - SP --->  1.0
Balanced Accuracy --->  0.9994011976047904
Jaccard Index --->TP/TP+FN+FP  0.9988023952095808


In [19]:
import numpy as np
from sklearn import metrics
auc = metrics.roc_auc_score(y_test, pred1)
print(auc)

0.9994011976047904
